This notebook is for looking through and accumulating potential image training data.

Download layer: 'CAL FIRE Timber Harvesting Plans TA83' from https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/CAL_FIRE_Timber_Harvesting_Plans_All_TA83/FeatureServer

In [ ]:
#Change to directory with .geojson file in it
my_path = "/content/drive/MyDrive/6. Spring 2024 Courses/DATA 450/"

##A) Package Installs & Imports

Install: ```pystac_client, planetary_computer, rioxarray contextily```

Import: ```pystac_client, planetary_computer, rioxarray as rio, rasterio, rasterio.plot.show, geopandas as gpd, matplotlib.pyplot as plt, shapely.get_coordinates(),  requests, IPython.display.Image, contextily as ctx```

In [ ]:
#install packages
!pip install pystac_client planetary_computer contextily rioxarray

In [ ]:
  #Import packages and get functions
import pystac_client
import planetary_computer
import rioxarray as rio
import contextily as ctx
import rasterio
from rasterio.plot import show

import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from shapely import get_coordinates
from shapely.geometry import shape

import requests
import time

from IPython.display import Image
from PIL import Image
from skimage import io, transform

##B) Import data and check geopandas dataframe
Below is an *summary statatisics* and *explanation of column names* about the numeric columns of the data. Once read in the data needs to be filtered for timber harvest plans that are not completed and plans not to harvest timber.

In [ ]:
timber = gpd.read_file(my_path+"CAL_FIRE_Timber_Harvesting_Plans_All_TA83_2639850418551769154.geojson")
 #after filters are applied, data to collect for training will be named 'harvested'

####1. column names and describe()

In [ ]:
timber.columns #column names

Index(['OBJECTID', 'GIS_ACRES', 'REGION', 'THP_YEAR', 'THP_NUM', 'COUNTY',
       'TIMBEROWNR', 'LANDOWNER', 'SILVI_1', 'SILVI_2', 'SILVI_CAT', 'YARD',
       'UNIT', 'PLAN_STAT', 'APPROVED', 'COMPLETED', 'COMMENTS', 'SPATL_MOD',
       'MODIFIED', 'HD_NUM', 'GLOBALID', 'geometry'],
      dtype='object')

In [ ]:
timber.describe()

,OBJECTID,GIS_ACRES,REGION,THP_YEAR,THP_NUM
count,73433.000000,73433.000000,73433.000000,73433.000000,73433.000000
mean,47166.644479,20.265127,1.604251,2015.425109,74.850381
std,29396.816177,109.816744,0.847445,4.002237,52.182437
min,1.000000,0.000519,1.000000,2009.000000,1.000000
25%,22948.000000,1.413363,1.000000,2012.000000,31.000000
50%,45264.000000,6.377017,1.000000,2015.000000,69.000000
75%,66647.000000,18.795057,2.000000,2019.000000,108.000000
max,116896.000000,8938.382409,4.000000,2023.000000,224.000000


####2. explaination of column names

1.  ```OBJECTID``` - Internal feature number: Sequential unique whole numbers that are automatically generated.

2.  ```GIS_ACRES```, GIS-calculated acreage

3.  ```REGION```, Administrative area:
```
[1: Coast, 2: Cascade, 3: South, 4: Sierra]
```
4.  ```THP_YEAR```, Year Timber Harvesting Plan (THP) submitted

5.  ```THP_NUM```, Timber Harvest Plan Number, Note: THP number starts from 1 at the beginning of every year.

6.  ```COUNTY```, County Code (County Name):
```
ALA (Alameda), ALP (Alpine), AMA (Amador), BUT (Butte), CAL (Calaveras), COL (Colusa), CCA (Contra Costa),
DEL (Del Norte), ELD (El Dorado), FRE (Fresno), GLE (Glenn), HUM (Humboldt), IMP (Imperial), INY (Inyo),
KER (Kern), KIN (Kings), LAK (Lake), LAS (Lassen), LAN (Los Angeles), MAD (Madera), MAN (Marin),
MAR (Mariposa), MEN (Mendocino), MER (Merced), MOD (Modoc), MOO (Mono), MON (Monterey), NAP (Napa),
NEV (Nevada), ORA (Orange), PLA (Placer), PLU (Plumas), RIV (Riverside), SAC (Sacramento), SBO (San Benito),
SBR (San Bernardino), SDO (San Diego), SFO (San Francisco), SJN (San Joaquin), SLO (San Luis Obispo),
SMO (San Mateo), SBA (Santa Barbara), SCL (Santa Clara), SCR (Santa Cruz), SHA (Shasta), SIE (Sierra),
SIS (Siskiyou), SOL (Solano), SON (Sonoma), STA (Stanislaus), SUT (Sutter), TEH (Tehama), TRI (Trinity),
TUL (Tulare), TUO (Tuolumne), VEN (Ventura), YOL (Yolo), YUB (Yuba)
```
7.  ```TIMBEROWNR```, Timber owner of record

8.  ```LANDOWNER```, Land owner of record

9.  ```SILVI_1```, Silvicultural Prescription:
```
Clearcut, Seed Tree Seed Step, Seed Tree Removal Step, Seed Tree Rem/Commercial Thin, Shelterwood Prep Step,
Shelterwood Seed Step, Shelterwood Removal Step, Shelterwood Rem/Commercial Thin, Selection, Group Selection,
Transition, Commercial Thin, Sanitation Salvage, Aspen/Meadow/Wet Area Restoration, Fuelbreak/Defensible Space,
Non Standard Practice, Oak Woodland Management, Rehabilitation of Understocked, Special Treatment Area,
Substantially Damaged Timberland, Variable Retention, Conversion
```
10. ```SILVI_2```, Alternative silvicultural prescription (the most nearly appropriate or feasible silvicultural method in California Forest Practice Rules and Act) or the secondary silvicultural prescription
```
Clearcut, Seed Tree Seed Step, Seed Tree Removal Step, Seed Tree Rem/Commercial Thin, Shelterwood Prep Step,
Shelterwood Seed Step, Shelterwood Removal Step, Shelterwood Rem/Commercial Thin, Selection, Group Selection,
Transition, Commercial Thin, Sanitation Salvage, Hardwood Release, Rehabilitation of Understocked,
Special Treatment Area, Variable Retention
```

11. ```SILVI_CAT```, Silvicultural Category:
```
Evenaged Management 14 CCR 913.1 [933.1, 953.1],
Unevenaged Management 14 CCR 913.2 [933.2, 953.2],
Intermediate Treatments 14 CCR 913.3 [933.3, 953.3],
Special Prescriptions and Other Management 14 CCR 913.4 [933.4, 953.4]
Timberland Conversion 14 CCR SUBCHAPTER 7, Article 7
Others: Road Right-of-Way, No Harvest Area
```
12. ```YARD```, Yarding method
```
Ground Based: [Tractor or Skidder],
Cable: [Cable System],
Combination: [Tractor with Cable option, Cable with Tractor option, Tractor with Helicopter option,
Cable with Helicopter option, Helicopter with Tractor option, Helicopter with Cable option,
Tractor with Cable and Helicopter option, Cable with Tractor and Helicopter option],
Other: [Balloon or Helicopter Animal Other]
```
13. ```UNIT```, Name or number of unit if mapped

14. ```PLAN_STAT```, Administrative status of THP
```
[Approved, Completed, Unlogged, Withdrawn]
```
15. ```APPROVED```, Date THP approved

16. ```COMPLETED```, Date THP completed or administratively closed out

17. ```COMMENTS```, Supplementary information:
```
Examples:
Am 9 change TO/TLO (Per Amendment 9, timber owner and timberland owner were changed.)
CCSTA (Coastal Commission Special Treatment Area)
Elk Creek STA (Elk Creek Special Treatment Area)
WLPZ (Watercourse and Lake Protection Zone)
```
18. ```SPATL_MOD```, Source of spatial modifications
       
19. ```MODIFIED```, Date of spatial modification

20. ```HD_NUM```, Timber Harvesting Document Number or Plan Number
```
'{REGION}-{THP_YEAR}-{THP_NUM}-{COUNTY}' = 'HD_NUM'
```
21. ```GLOBALID```,

22. ```geometry``` Polygon object decribing harvest area

###3. Convert CRS and filter out data
Bellow I filter out data that is not completed, data that is not labeled even-aged or uneven-aged, and data of stands smaller than 15 acres. Since the NAIP started in 2011 and is only done every two years to be safe I only included data from 2010 to 2018 to prevent as many empty searchs as possible. Then there is a breif look at how the sample size changed.

In [ ]:
timber['COMP_YEAR'] = timber['COMPLETED'].str.split().str[3] #get Year completed for later use
timber = timber.loc[timber['PLAN_STAT'] == 'Completed']     #make sure I only have info for completed harvests
timber['COMP_YEAR_int'] = timber['COMP_YEAR'].astype('int32') #store year as int for later

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
filtered_gdf = timber[(timber['THP_YEAR'] >= 2011) # only want info for harvest approved and completed
                    & (timber['COMP_YEAR_int'] <= 2018)  #between 2011 - 2018
                    & (timber['GIS_ACRES'] > 14.9) # only want larger stands to look at
                    & ((timber['SILVI_CAT']=='Evenaged Management') # and keep only even-aged
                    | (timber['SILVI_CAT'] == 'Unevenaged Management'))] #or uneven-aged management plans

In [ ]:
# Convert to EPSG:4326 (WGS84 - latitude and longitude)
harvested = filtered_gdf.to_crs(epsg=4326).reset_index(drop=True)

In [ ]:
harvested.head() #make sure new columns exist and polygon gemtery in correct CRS

,OBJECTID,GIS_ACRES,REGION,THP_YEAR,THP_NUM,COUNTY,TIMBEROWNR,LANDOWNER,SILVI_1,SILVI_2,...,APPROVED,COMPLETED,COMMENTS,SPATL_MOD,MODIFIED,HD_NUM,GLOBALID,geometry,COMP_YEAR,COMP_YEAR_int
0,11,41.942087,1,2011,98,SON,Soper Co,Soper Co,Group Selection,None,...,"Fri, 13 Jul 2012 07:00:00 GMT","Tue, 06 Mar 2018 08:00:00 GMT",None,None,None,1-11-098-SON,e75eb4e3-778d-4b51-ae07-b70562fc2622,"POLYGON ((-123.21534 38.51008, -123.21557 38.5...",2018,2018
1,23,92.841250,1,2011,98,SON,Soper Co,Soper Co,Selection,Special Treatment Area,...,"Fri, 13 Jul 2012 07:00:00 GMT","Tue, 06 Mar 2018 08:00:00 GMT",None,None,None,1-11-098-SON,9817ddb3-8d51-48bf-8b1c-db73149ba9e9,"POLYGON ((-123.20231 38.51064, -123.20164 38.5...",2018,2018
2,44,60.454129,1,2011,102,SCR,Big Creek Lumber Co,Big Creek Lumber Co,Selection,None,...,"Wed, 21 Aug 2013 07:00:00 GMT","Wed, 22 Jun 2016 07:00:00 GMT",None,None,None,1-11-102-SCR,05f8afe1-e2a1-4d5a-99a9-24fdc161c8bb,"POLYGON ((-122.11731 37.15019, -122.11687 37.1...",2016,2016
3,45,104.172893,1,2013,27,SCR,CA Soquel Demonstration State Forest,CA Soquel Demonstration State Forest,Selection,None,...,"Tue, 03 Dec 2013 08:00:00 GMT","Thu, 20 Oct 2016 07:00:00 GMT",None,None,None,1-13-027-SCR,ba1212dd-ba12-4d67-b5a7-d8adaff96057,"POLYGON ((-121.94056 37.09343, -121.94040 37.0...",2016,2016
4,57,155.366575,2,2011,30,PLU,Collins Pine Co,Collins Pine Co,Group Selection,None,...,"Tue, 22 Nov 2011 08:00:00 GMT","Mon, 22 Dec 2014 08:00:00 GMT",None,None,None,2-11-030-PLU,3cc33812-960c-4f74-a50b-83d010510830,"POLYGON ((-121.28037 40.33250, -121.27464 40.3...",2014,2014


####use len({gdf}) to compare size of filtered gdfs

In [ ]:
harvested_len = len(harvested)     #number of rows in 'harvested'
timber_len = len(timber)    #number of rows in 'timber'
print(f"After filtering 'harvested': {harvested_len} rows (plans)\nBefore filtering 'timber': {timber_len} rows")

After filtering 'harvested': 3457 rows (plans)
Before filtering 'timber': 33749 rows


##C) Custom Functions

Functions to get ```area_of_interest```, ```range_before```, ```range_after```, ```item_before```, ```item_after```, ```before_clipped```, ```after_clipped```, and save the ```..._clipped```'s to .tif files

###1. Create desired area of interest object
```create_area_of_interest()``` function

*Purpose*:
Prepares coordinates to pass to ```pystac_client....search()```

*Input/Output*:
Takes geometry object and returns object with ```dtype="polygon" or "multipolygon"``` and ```coordinates=[[[{list(coords)}]]```

In [ ]:
def create_area_of_interest(polygon_geometry):
    # Access the coordinates of the exterior ring of the polygon as a list of tuples
    exterior_coordinates = list(polygon_geometry.exterior.coords)
    # Check if the polygon has interior rings (holes)
    if polygon_geometry.interiors:
      # Access the coordinates of the interior rings of the polygon as a list of lists of tuples
      interior_coordinates = [list(interior.coords) for interior in polygon_geometry.interiors]
      interior_coordinates = [[list(coord) for coord in interior] for interior in interior_coordinates]
      # Create the area_of_interest (Multipolygon)
      area_of_interest = {
        "type": "MultiPolygon",
        "coordinates": [[exterior_coordinates] + interior_coordinates]
      }
    else:
       # Create the area_of_interest (Polygon)
      area_of_interest = {
        "type": "Polygon",
        "coordinates": [exterior_coordinates]
      }

    return area_of_interest

###2. Create search date ranges
```get_search_range()``` function

*Purpose*:
Creates date range to pass to ```pystac_client....search()```

*Input/Output*: Year string in the form: ```'%Y'``` and returns a range of dates in the form: ```'%Y-01-01/%Y-12-31'``` the second date in the range's year being offset by ```year_offset```

In [ ]:
#month_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

def get_search_range(date_str, year_offset):
    year = date_str #grab %Y
    yearout = str(int(year) + year_offset) #add 2 years to %Y

    range = f"{year}-01-01/{yearout}-12-31" #put in form '{%Y-%m-%d}/{%Y-%m-%d}' ({original}/{modified})

    return range

###3. Search of the plantary computer's catalog
```perform_search()``` function

*Purpose*: Use ```pystac_client....search()``` for a photo that overlaps the ```area_of_interest```, to sort the results from most ```area_of_overlap``` to least. It also needs to handle no search results.

*Input/Output*:
Takes ```catalog```, ```area_of_interest```, and ```range``` and returns the first Item in the sorted list of results, if the item exists.


In [ ]:
def perform_search(catalog, area_of_interest, range):
    #search catalog for area of interest in date range
    search = catalog.search(
        collections=["naip"], intersects=area_of_interest, datetime=range
    )

    #store result as list of items
    items = search.item_collection()
    print(f"{len(items)} Items found in range")

    area_shape = shape(area_of_interest) #define shape of area of interest
    target_area = area_shape.area #target area is shape of area of interest

    #desired method to sort
    def area_of_overlap(item):
      overlap_area = shape(item.geometry).intersection(shape(area_of_interest)).area #from shape of item calc interstion between image and area of interest
      return overlap_area / target_area

    #sort items by most inersected area
    item = sorted(items, key=area_of_overlap, reverse=True)


    if item:#return first item in list of items
      return item[0]

###4. Clip found images to shape of THP
```clip_images_to_shape()``` function

*Purpose*: Trim the images to the size of the harvest plan's ```area_of_interest```.

*Input/Output*: Takes search ```item```, selects the image, and ```area_of_interest``` and returns a clipped image to the size of the intended harvest plans geometry.

In [ ]:
def clip_images_to_shape(item, area_of_interest):
    #data set of image bands
    ds = rio.open_rasterio(item.assets["image"].href).sel(band=[1, 2, 3, 4])

    #correct CRS of image bands
    ds_reprojected = ds.rio.reproject("EPSG:4326", resampling=0)

    #clip image bands
    clipped = ds_reprojected.rio.clip([area_of_interest])

    return clipped

###5. Save images as tif files
```save_to_tif()``` function

*Purpose*: Save images as .tif files and label according to if the image is from the ```before``` or ```after``` group. And to help group the same areas images it started with the unique row id (```OBJECTID```).

*Input/Output*:
Takes a before and an after image and ```io.imsave```'s each band as .tif's and labels each .tif according to ```{OBJECTID}_{data.properties['datetime']}_{prefix}.tif```

```{OBJECTID}```: unique id

```{prefix}```: ["before", "after"]

```{data.properties['datetime']}```: date image was taken

In [ ]:
image_file_path = "/content/drive/MyDrive/Emily/Images/" #change to where you want the images to go

def save_to_tif(before, after):
    # For every band run this loop (1-4, inclusive)
    for data, prefix in [(before, "before"), (after, "after")]:
        # Extracting individual bands and normalizing pixel values
        red_array = np.array(data.sel(band=1)).astype(np.float32) / 255.0
        green_array = np.array(data.sel(band=2)).astype(np.float32) / 255.0
        blue_array = np.array(data.sel(band=3)).astype(np.float32) / 255.0
        nir_array = np.array(data.sel(band=4)).astype(np.float32) / 255.0

        # Stack the bands into an RGB image
        rgb_image = np.dstack((nir_array, red_array, green_array))

        # Convert the numpy array to PIL image
        rgb_image = Image.fromarray((rgb_image * 255).astype(np.uint8))

        # Save the image
        rgb_image.save(f"{image_file_path}{harvested.OBJECTID[rows]}_{harvested.SILVI_1[rows]}_{prefix}.tif")
        print(f"\t\t{prefix} image saved")


##D) NAIP Catalog

Run cell to get api token access (```catalog```)

**Data access**

The datasets hosted by the Planetary Computer are available from Azure Blob Storage. We'll use pystac-client to search the Planetary Computer's STAC API for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a modifier so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See Reading from the STAC API and Using tokens for data access for more.

source: https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/naip/naip-example.ipynb

In [ ]:
#access plantetary computer catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

#### loop to generate training data

In [ ]:
even_index = harvested.index[harvested['SILVI_CAT'] == 'Evenaged Management'].tolist() #1
harvested.loc[even_index[747]]

OBJECTID                                                     20725
GIS_ACRES                                                26.342702
REGION                                                           2
THP_YEAR                                                      2015
THP_NUM                                                          4
COUNTY                                                         SHA
TIMBEROWNR                          Shasta Forests Timberlands LLC
LANDOWNER                           Shasta Forests Timberlands LLC
SILVI_1                                                   Clearcut
SILVI_2                                                       None
SILVI_CAT                                      Evenaged Management
YARD                                            Tractor or Skidder
UNIT                                                          None
PLAN_STAT                                                Completed
APPROVED                             Tue, 30 Jun 2015 07:00:00

In [ ]:
uneven_index = harvested.index[harvested['SILVI_CAT'] == 'Unevenaged Management']
harvested.loc[uneven_index[747]]

In [ ]:
print("Uneven-Aged Images Beginning")
num = 0
my_row = even_index[num:]
i=num-1
for rows in my_row:
  i = i+1
  print(f"Run #{i} index: {rows}, objectID: {harvested.OBJECTID[rows]}")
  polygon_geometry = harvested.loc[rows].geometry

  area_of_interest = create_area_of_interest(polygon_geometry) #Get THP's shape information

  offset = 2                                      #we'll search the approval/completed dates + 3 years out
  approved = str(harvested['THP_YEAR'].loc[rows])      #date THP was 'Approved'
  completed = harvested['COMP_YEAR'].loc[rows]    #date RM-71 was accepted and THP was offically 'Completed'

  range_old = get_search_range(approved, offset)  #approval date + 3 years
  range_new = get_search_range(completed, offset) #completed date + 3 years

  item_older = perform_search(catalog, area_of_interest, range_old)
  time.sleep(5) #wait 5 seconds between searchs
  item_newer = perform_search(catalog, area_of_interest, range_new)

  #SAVE IMAGE BANDS AS .TIF
  if item_older and item_newer and (item_older.properties['datetime'] < item_newer.properties['datetime']):
    #clip image bands
    old_clipped = clip_images_to_shape(item_older, area_of_interest)
    new_clipped = clip_images_to_shape(item_newer, area_of_interest)

    save_to_tif(old_clipped, new_clipped)
  time.sleep(5) #wait 5 seconds between searchs

Run #1056 index: 1709, objectID: 37185
2 Items found in range
4 Items found in range
		before image saved
		after image saved
Run #1057 index: 1714, objectID: 37668
2 Items found in range
2 Items found in range
		before image saved
		after image saved
Run #1058 index: 1725, objectID: 38089
2 Items found in range
2 Items found in range
		before image saved
		after image saved
Run #1059 index: 1731, objectID: 38170
1 Items found in range
1 Items found in range
		before image saved
		after image saved
Run #1060 index: 1734, objectID: 38261
1 Items found in range
2 Items found in range
		before image saved
		after image saved
Run #1061 index: 1738, objectID: 38437
2 Items found in range
2 Items found in range


In [ ]:
print("Uneven-Aged Images Beginning")
num = 0
my_row = uneven_index[num:]
i=num-1
for rows in my_row:
  i = i+1
  print(f"Run #{i} index: {rows}, objectID: {harvested.OBJECTID[rows]}")
  polygon_geometry = harvested.loc[rows].geometry

  area_of_interest = create_area_of_interest(polygon_geometry) #Get THP's shape information

  offset = 2                                      #we'll search the approval/completed dates + 3 years out
  approved = str(harvested['THP_YEAR'].loc[rows])      #date THP was 'Approved'
  completed = harvested['COMP_YEAR'].loc[rows]    #date RM-71 was accepted and THP was offically 'Completed'

  range_old = get_search_range(approved, offset)  #approval date + 3 years
  range_new = get_search_range(completed, offset) #completed date + 3 years

  item_older = perform_search(catalog, area_of_interest, range_old)
  time.sleep(5) #wait 5 seconds between searchs
  item_newer = perform_search(catalog, area_of_interest, range_new)

  #SAVE IMAGE BANDS AS .TIF
  if item_older and item_newer and (item_older.properties['datetime'] < item_newer.properties['datetime']):
    #clip image bands
    old_clipped = clip_images_to_shape(item_older, area_of_interest)
    new_clipped = clip_images_to_shape(item_newer, area_of_interest)

    save_to_tif(old_clipped, new_clipped)
  time.sleep(5) #wait 5 seconds between searchs

##E) Experiemental Section

Set desired rows to inspect, get ranges and area of interest, apply to a search, compare found results and veiw full image

###1. Timber harvest treatment (prescriptions) and selecting my_row
CalFire specifed ~8 categories of prescriptions they are interestes in being able to detect remotely: Clearcuts, Seed Tree Removal Step, Shelterwood Removal Step, Selection, Group Selection, Commercial Thin, Fuelbreak/Defensible Space, Rehabilitation of Understocked.

In this *Experimental* section you may choose to look at a single harvest plan at a time and compare before and after images.

In [ ]:
#Creating lists of indexs in 'harvested' that corresponds to specific presriptions

#SILVI_CAT: Even-Aged Management
even_index = harvested.index[harvested['SILVI_CAT'] == 'Evenaged Management'].tolist() #1

clear_index = harvested.index[harvested['SILVI_1'] == 'Clearcut'].tolist()
st_removal_index = harvested.index[harvested['SILVI_1'] == 'Seed Tree Removal Step'].tolist()
sh_index = harvested.index[harvested['SILVI_1'] == 'Shelterwood Removal Step'].tolist()

#SILVI_CAT: Uneven-Aged Management
uneven_index = harvested.index[harvested['SILVI_CAT'] == 'Unevenaged Management'] #2

select_index = harvested.index[harvested['SILVI_1'] == 'Selection'].tolist()
group_index = harvested.index[harvested['SILVI_1'] == 'Group Selection'].tolist()

#SILVI_CAT: Intermediate Managemenet
#thin_index = harvested.index[harvested['SILVI_1'] == 'Commercial Thin'].tolist()

#SILVI_CAT: Special Prescription & Other Management
#fuelbreak_defensible_space_index = harvested.index[harvested['SILVI_1'] == 'Fuelbreak/Defensible Space'].tolist()
#rehab_understocked_index = harvested.index[harvested['SILVI_1'] == 'Rehabilitation of Understocked'].tolist()

print(f"Even-aged Management: \t\t\t{len(even_index)}")
print(f"Uneven-aged Management: \t\t{len(uneven_index)}\n")

print(f"Clearcuts:\t\t\t\t{len(clear_index)}")
print(f"Seed Tree Removal Step:\t\t\t{len(st_removal_index)}")
print(f"Shelterwood Removal Step:\t\t{len(sh_index)}")
print(f"Selection:\t\t\t\t{len(select_index)}")
print(f"Group Selection:\t\t\t{len(group_index)}")
#print(f"Commercial Thin:\t\t\t{len(thin_index)}")
#print(f"Fuelbreak/Defensible Space:\t\t{len(fuelbreak_defensible_space_index)}")
#print(f"Rehabilitation of Understocked:\t\t{len(rehab_understocked_index)}")

Even-aged Management: 			2217
Uneven-aged Management: 		1240

Clearcuts:				1416
Seed Tree Removal Step:			60
Shelterwood Removal Step:		88
Selection:				524
Group Selection:			518
Commercial Thin:			0
Fuelbreak/Defensible Space:		0
Rehabilitation of Understocked:		0


Above you can see the total of each prescription in ```'harvested'```. To look at a specific row: you just pick any number smaller than the prescriptions total and give it to ```my_row```

```
#e.g. one row of clearcuts
my_row = clear_index[8190]
harvested.loc[my_row]
```
But if you want to look at every image of a entire prescription just pass the index itself
```
#e.g. all clearcuts
my_row = clear_index
for row in my_row:
  harvested.loc[row]
```

####my_row =

In [ ]:
print(harvested.index[harvested['OBJECTID'] == 30793])
print(even_index[881])

Index([1434], dtype='int64')
1434


In [ ]:
my_row = even_index[881] #harvest plan we want to look at

if isinstance(my_row, list):
  for row in my_row:
    print(harvested['SILVI_1'].loc[row])
else:
  print(harvested.loc[my_row]) #print info

OBJECTID                                                     30793
GIS_ACRES                                                19.910694
REGION                                                           1
THP_YEAR                                                      2012
THP_NUM                                                        105
COUNTY                                                         HUM
TIMBEROWNR                               Green Diamond Resource Co
LANDOWNER                                Green Diamond Resource Co
SILVI_1                                                   Clearcut
SILVI_2                                                       None
SILVI_CAT                                      Evenaged Management
YARD                                            Tractor or Skidder
UNIT                                                          None
PLAN_STAT                                                Completed
APPROVED                             Fri, 05 Apr 2013 07:00:00

###2. Area of interest
This section creates an object we can pass to the plantary computer API. We need the geometry from our data set in a particular format to pass to the search.

In [ ]:
harvested.loc[my_row].geometry

AttributeError: 'MultiPolygon' object has no attribute 'coord'

In [ ]:
rows = my_row
area_of_interest = create_area_of_interest(harvested.loc[my_row].geometry) #Get THP's shape information

####double check form

In [ ]:
area_of_interest #double check form

###3. Search Ranges
We are interested in lookin at a ```before``` and ```after``` picture of the area that was harvested. We know the timber was approved to be harvested and when the land/timber owner told us it was done. Therefore, our ```'Approved'``` and ```'Completed'``` Years from the dataset serve as a starting point for setting each prospective range. Since the NAIP is completed in California every 2 years during the summer, the search range was 2 years out from the original date.

In [ ]:
offset = 2 #we'll search the approval/completed dates + 3 years out

approved = str(harvested['THP_YEAR'].loc[my_row]) #date THP was approved
completed = harvested['COMP_YEAR'].loc[my_row] #date RM-71 was accepted and THP was offically marked as completed

range_before = get_search_range(approved, offset) #approval date + 3 years
range_after = get_search_range(completed, offset) #completed date + 3 years

#check ranges are normal
print(f"range before harvest: {range_before} \nrange after harvest: {range_after}")

range before harvest: 2013-01-01/2015-12-31 
range after harvest: 2015-01-01/2017-12-31


###4. Search
Take ```area_of_interest``` and desired ```range``` and search for NAIP images using planatray computer's API. (Make sure token cell was run in NAIP Catalog tab)

In [ ]:
print(f"Before Harvest:")
item_before = perform_search(catalog, area_of_interest, range_before)
time.sleep(5) #wait 5 seconds between searchs
print(f"After Harvest:")
item_after = perform_search(catalog, area_of_interest, range_after)
#time.sleep(5) #wait 5 seconds between searchs

Before Harvest:
2 Items found in range
After Harvest:
2 Items found in range


###5. Compare Image and Harvest Plan Dates
* Are we getting images from too far out?
* Do the ranges cross over?
* If so, are the images from the same date?

In [ ]:
item_after

In [ ]:
#Approval Image
print(f"Search Date Range:\t{range_before}\nApproval Date:\t\t{approved}\nImage Date:\t\t{item_before.properties['datetime']}")

Search Date Range:	2013-01-01/2015-12-31
Approval Date:		2013
Image Date:		2014-06-07T00:00:00Z


In [ ]:
#Completed Image
print(f"Search Date Range:\t{range_after}\nCompletion Date:\t{completed}\nImage Date:\t\t{item_after.properties['datetime']}")

Search Date Range:	2015-01-01/2017-12-31
Completion Date:	2015
Image Date:		2016-05-28T00:00:00Z


In [ ]:
#Check if you should continue
if item_before.properties['datetime'] == item_after.properties['datetime']:
  print(f"Error: Pictures are from same day")
else:
  print(f"Continue")

Continue


###6. Display Full Images

In [ ]:
#View image before timber harvest completed
Image(url=item_before.assets["rendered_preview"].href)

In [ ]:
#View image after timber harvest completed
Image(url=item_after.assets["rendered_preview"].href)

##F) Image Clips
View GSD, select image's RGB and NIR bands, reproject coordinates (they come in a differnet format than you searched with), clip image to area_of_interest, view image once clipped, view bands of clipped images

In [ ]:
#What are the before and after images' ground sampling distances(GSD)?
print(f"Ground Sampling Distance of Images")
print(f"Before Image GSD: {item_before.properties['gsd']}\nAfter Image GSD: {item_after.properties['gsd']}")

###1. Get clipped images

In [ ]:
#if images before and after were found and item_after comes after item_before
if item_before and item_after and (item_before.properties['datetime'] < item_after.properties['datetime']):
  #clip before and after images
    before_clipped = clip_images_to_shape(item_before, area_of_interest)
    after_clipped = clip_images_to_shape(item_after, area_of_interest)

In [ ]:
before_clipped

In [ ]:
save_to_tif(before = before_clipped, after = after_clipped)

		before image saved
		after image saved


###2. View Full Clipped Images

In [ ]:
#view clipped before image of THP
before_clipped.plot.imshow(robust=True, figsize=(10, 10))
plt.show()

In [ ]:
#view clipped after image of THP
after_clipped.plot.imshow(robust=True, figsize=(10, 10))
plt.show()

###3. View Clipped Image Bands
View prescription, intended geometry, and before & after images RGB & NIR bands side by side

In [ ]:
#What row and treatment are we currrently looking at?
treatment_plan = harvested['SILVI_1'].iloc[my_row]
print(f"Row {my_row}'s Treatment Plan: {treatment_plan}")

In [ ]:
#How is the picture supposed to be shaped?
harvested.loc[my_row].geometry #shape of geometry

####Band 1: Red

In [ ]:
fig, (axR1, axR2) = plt.subplots(1, 2) #show them next to eachother
#Before
axR1.imshow(before_clipped.sel(band=1).values)
axR1.set_title('Before')
#After
axR2.imshow(after_clipped.sel(band=1).values)
axR2.set_title('After')

plt.tight_layout() #plot close together but no overlap
plt.show() #look!

####Band 2: Blue

In [ ]:
fig, (axB1, axB2) = plt.subplots(1, 2)

axB1.imshow(before_clipped.sel(band =2).values)
axB1.set_title('Before')

axB2.imshow(after_clipped.sel(band =2).values)
axB2.set_title('After')

plt.tight_layout()
plt.show()

####Band 3: Green

In [ ]:
fig, (axG1, axG2) = plt.subplots(1, 2)

axG1.imshow(before_clipped.sel(band =3).values)
axG1.set_title('Before')

axG2.imshow(after_clipped.sel(band =3).values)
axG2.set_title('After')

plt.tight_layout()
plt.show()

####Band 4: Near-InfraRed

In [ ]:
fig, (axNIR1, axNIR2) = plt.subplots(1, 2)

axNIR1.imshow(before_clipped.sel(band=4).values)
axNIR1.set_title('Before')

axNIR2.imshow(after_clipped.sel(band=4).values)
axNIR2.set_title('After')

plt.tight_layout()
plt.show()

##G) Data Visualization (Maps)
General Info about our Data Set

Using original CRS to plot (not using gdf: ```'harvested'```)

In [ ]:
#completed and harvestable plans (filtered plans - original CRS)
filtered_gdf.plot(figsize=(8, 8))

In [ ]:
#North West Section of California
NorWestCal = filtered_gdf.cx[:-200000, 200000:]

NorWestCal.plot(figsize=(10, 10))

In [ ]:
#NorthWest California by Treatment Category w/ Base Map

silvicatplot = NorWestCal.plot(column='SILVI_CAT', cmap='tab20c', legend=True, figsize=(20, 20), legend_kwds={'bbox_to_anchor': (1, 1), 'loc': 'upper left'})

ctx.add_basemap(silvicatplot, crs=NorWestCal.crs,source=ctx.providers.CartoDB.Positron)
plt.show()

In [ ]:
#NorthWest California by Treatment Type w/ Base Map
fig, ax = plt.subplots(figsize=(20, 20))

silvi1plot=NorWestCal.plot(column='SILVI_1', cmap='tab20', legend=True, ax=ax, legend_kwds={'bbox_to_anchor': (1, 1), 'loc': 'upper left'})
ctx.add_basemap(silvi1plot, crs=NorWestCal.crs,source=ctx.providers.CartoDB.Positron)
plt.show()

In [ ]:
#trying add_basemap()
ax = NorWestCal.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
ctx.add_basemap(ax, crs=NorWestCal.crs,source=ctx.providers.CartoDB.Positron)
ax.set_axis_off()